<a href="https://colab.research.google.com/github/mbalbi/ciencia_de_datos/blob/main/tps/2c2025/ICDIC_TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Práctico 3: Regresión Lineal Generalizada

**Objetivos:** Con este trabajo práctico se busca que los alumnos completen la ejercitación en modelos de regresión lineal, extendiendo el proceso de inferencia a modelos que difieran del tradicional de errores normales. Esto se hará a partir de 2 ejercicios de regresión categórica.

**Librerías:** En este trabajo se utilizará

*   Numpy: matemática básica y definición de matrices y vectores; simulación de variables aleatorias
*   Seaborn: construcción de gráficos
*   Bambi: Librería basada en PyMC para la inferencia estadística utilizando cadenas de Markov

También necesitaremos de la librería ArviZ para ver los resultados de nuestra inferencia.

In [ ]:
# Instalación e import de Bambi
try:
  import bambi as bmb
except:
  !pip install bambi
  import bambi as bmb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import pandas as pd
import io

# para registrar el tiempo de corrida
import datetime

In [ ]:
start_datetime = datetime.datetime.now()

print('Inicio de corrida: ',start_datetime)

Inicio de corrida:  2025-11-17 15:33:03.534309


Grupo:

Integrantes:

*   Apellido, Nombres XXXXXX
*   Apellido, Nombres XXXXXX
*   Apellido, Nombres XXXXXX
*   Apellido, Nombres XXXXXX
*   Apellido, Nombres XXXXXX

## Ejercicio 1: Clasificación de aguas superficiales de México

En este ejercicio vamos a proponer un modelo de regresión para clasificar las aguas superficiales de los principales cuerpos superficiales de México, según su calidad.

### Introducción

La Comisión Nacional del Agua (CONAGUA), a través de la Red Nacional de Medición de Calidad del Agua, realiza el monitoreo de los principales cuerpos de agua de México en sus más de 1723 sitios de muestreo. Los datos que utilizaremos fueron obtenidos desde el siguiente link: https://www.gob.mx/conagua/articulos/calidad-del-agua

### Carga del Dataset

Empecemos abriendo el archivo en Google Colab

In [ ]:
#Abrimos un archivo local, es decir, que ha sido descargado en su computadora
from google.colab import files
uploaded = files.upload()

Saving aguas_superficiales.csv to aguas_superficiales.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['aguas_superficiales.csv']))
df.head()

,Unnamed: 0,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,CUENCA,CUERPO DE AGUA,TIPO,SUBTIPO,...,OD_PORC_MED,OD_PORC_FON,TOX_D_48_UT,TOX_V_15_UT,TOX_D_48_SUP_UT,TOX_D_48_FON_UT,TOX_FIS_SUP_15_UT,TOX_FIS_FON_15_UT,SEMAFORO,GRUPO
0,0,CARMINA 2,CARMINA 2,RÍO BRAVO,COAHUILA DE ZARAGOZA,ACUÑA,RÍO BRAVO 5,DATO PENDIENTE,LÓTICO,ARROYO,...,NaN,NaN,<1,<1,NaN,NaN,NaN,NaN,Amarillo,LOTICO
1,1,DLAGU0001RNL21,PRESA NATILLAS DE ABAJO,LERMA SANTIAGO PACÍFICO,AGUASCALIENTES,COSIO,RÍO SAN PEDRO,PRESA NATILLAS DE ABAJO,LÉNTICO,PRESA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rojo,LENTICO
2,2,DLAGU0002RNL21,PRESA EL JOCOQUI,LERMA SANTIAGO PACÍFICO,AGUASCALIENTES,RINCÓN DE ROMOS,RÍO SAN PEDRO,PRESA EL JOCOQUI,LÉNTICO,PRESA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verde,LENTICO
3,3,DLAGU0003RNL21,BORDO SANTA ELENA CORTINA,LERMA SANTIAGO PACÍFICO,AGUASCALIENTES,AGUASCALIENTES,PRESA EL NIÁGARA,PRESA SANTA ELENA,LÉNTICO,PRESA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rojo,LENTICO
4,4,DLAGU0004RNL21,HUMEDAL BUENAVISA DE PEÑUELAS CHARCA NO. 5,LERMA SANTIAGO PACÍFICO,AGUASCALIENTES,AGUASCALIENTES,PRESA AJOJUCAR,HUMEDAL BUENAVISA,LÉNTICO,HUMEDAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rojo,LENTICO


Observemos las columnas del dataset:

In [ ]:
df.columns

Index(['Unnamed: 0', 'CLAVE', 'SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO',
       'MUNICIPIO', 'CUENCA', 'CUERPO DE AGUA', 'TIPO', 'SUBTIPO', 'LONGITUD',
       'LATITUD', 'PERIODO', 'DBO_mg/L', 'DQO_mg/L', 'SST_mg/L',
       'COLI_FEC_NMP_100mL', 'E_COLI_NMP_100mL', 'ENTEROC_NMP_100mL',
       'OD_PORC', 'OD_PORC_SUP', 'OD_PORC_MED', 'OD_PORC_FON', 'TOX_D_48_UT',
       'TOX_V_15_UT', 'TOX_D_48_SUP_UT', 'TOX_D_48_FON_UT',
       'TOX_FIS_SUP_15_UT', 'TOX_FIS_FON_15_UT', 'SEMAFORO', 'GRUPO'],
      dtype='object')

Este contiene, además de la información geográfica de las estaciones de muestreo, resultados de ensayos de calidad del agua, siendo sus variables:
- **Demanda Química de Oxígeno: DQO**
- **Sólidos Suspendidos Totales: SST**
- **Coliformes fecales: CF**
- **Escherichia coli: E_COLI**
- **Enterococos fecales: ENTEROC_FEC**
- **Porcentaje de saturación de Oxígeno Disuelto: OD**
- **Toxicidad aguda: TOX**

En base en estos indicadores, se clasifica el agua según su calidad, siguiendo una escala de tipo **semáforo** que considera 3 colores: **verde, amarillo y rojo**, correspondientes a buena calidad, aceptable y contaminada, respectivamente.

In [ ]:
# Cantidad de aguas de cada categoría:
df['SEMAFORO'].value_counts()

,count
SEMAFORO,
Verde,2326
Amarillo,2076
Rojo,1823


### Análisis Exploratorio de Datos

En primer lugar, vamos a analizar los datos y a limpiarlos, para luego desarrollar el modelo.

Nosotros trabajaremos con las aguas de tipo **lótico**. Además, conservaremos únicamente las columnas relacionadas con la calidad de agua (no los datos geográficos). Eliminaremos a su vez la DBO.

In [ ]:
df = df[df['GRUPO'] == 'LOTICO']

In [ ]:
# Selección de variables sobre calidad del agua de tipo lotico
df = df[['DQO_mg/L','SST_mg/L', 'COLI_FEC_NMP_100mL','E_COLI_NMP_100mL',
         'ENTEROC_NMP_100mL', 'OD_PORC', 'TOX_D_48_UT', 'TOX_V_15_UT', 'SEMAFORO']].copy()

# Renombrar columnas
df.rename(columns={'DQO_mg/L': 'DQO',
                    'SST_mg/L': 'SST',
                    'COLI_FEC_NMP_100mL': 'COLI',
                    'E_COLI_NMP_100mL': 'E_COLI',
                    'ENTEROC_NMP_100mL': 'ENTEROC',
                    'OD_PORC': 'OD',
                    'TOX_D_48_UT': 'TOX_D',
                    'TOX_V_15_UT': 'TOX_V'
                     },
            inplace=True)

#### Actividad 1

Utilizando `.info()`, vamos a analizar si el tipo de dato es el esperado: por ejemplo si la columna que debe contener textos, efectivamente sea de tipo `object` y las numéricas, `float` o `interger`, etc.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3372 entries, 0 to 6224
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DQO       3241 non-null   object
 1   SST       3218 non-null   object
 2   COLI      3234 non-null   object
 3   E_COLI    3159 non-null   object
 4   ENTEROC   43 non-null     object
 5   OD        3227 non-null   object
 6   TOX_D     3121 non-null   object
 7   TOX_V     3112 non-null   object
 8   SEMAFORO  3372 non-null   object
dtypes: object(9)
memory usage: 263.4+ KB


**Actividad**: Utilizando `replace()` haga las moficaciones que sean necesarias en los valores y convierta el tipo de dato.

NOTA: A fines del trabajo práctico, si un dato es indicado como `menor a X` asuma que vale directamente X.

In [ ]:
# Limpieza
# CODIGO DEL ALUMNO ############################################################






################################################################################
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3372 entries, 0 to 6224
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DQO       3241 non-null   float64 
 1   SST       3218 non-null   float64 
 2   COLI      3234 non-null   float64 
 3   E_COLI    3159 non-null   float64 
 4   ENTEROC   43 non-null     float64 
 5   OD        3227 non-null   float64 
 6   TOX_D     3121 non-null   float64 
 7   TOX_V     3112 non-null   float64 
 8   SEMAFORO  3372 non-null   category
dtypes: category(1), float64(8)
memory usage: 240.5 KB


**Pregunta:** ¿Existe una variable que no merece la pena incluir debido a la cantidad de datos?

**Respuesta:**

#### Actividad 2

El DataFrame `df_new` va a ser nuestro dataset de cabecera para hacer la inferencia. A continuación, mostramos distintas estadísticas de los potenciales predictores.

In [ ]:
df_new = df[['DQO', 'SST', 'COLI', 'E_COLI', 'OD', 'TOX_D', 'TOX_V', 'SEMAFORO']].copy()
df_new.dropna(inplace = True)

df_new.describe()

,DQO,SST,COLI,E_COLI,OD,TOX_D,TOX_V
count,3052.000000,3052.000000,3.052000e+03,3.052000e+03,3052.000000,3052.000000,3052.000000
mean,82.854117,114.408303,3.604824e+05,3.757695e+04,68.308882,2.788434,17.968823
std,295.516167,1120.535180,1.230317e+07,5.410151e+05,29.449573,26.203681,606.986305
min,10.000000,10.000000,3.000000e+00,3.000000e+00,10.000000,1.000000,1.000000
25%,14.037500,11.000000,1.376000e+03,1.450000e+02,50.137500,1.000000,1.000000
50%,26.582500,26.000000,3.873000e+03,8.315000e+02,73.550000,1.000000,1.000000
75%,57.237500,64.757500,2.400000e+04,7.800000e+03,91.700000,1.000000,1.822500
max,7900.000000,57880.000000,4.804210e+08,1.950598e+07,152.700000,1250.000000,33333.330000


**Actividad**: ¿Qué puede decir de la tabla anterior? Analice dispersiones, valores medios y extremos entre las variables, y rangos de valores. ¿Qué opciones hay para

- Recudir el rango de valores de una variable y reducir la asimetría, pasando de un plano positivo a valores en todo el dominio de los reales?
- Asgurar que los datos tengan todos escalas similares?

**Su Respuesta:**

### Datos de entrenamiento y de testeo

Antes de desarrollar los modelos, vamos a separar los datos en 2: uno para entrenar el modelo y el otro, para testearlo:

In [ ]:
import random

def split_data(df, train_size=0.8, random_state=42):
    random.seed(random_state)

    data = df.values.tolist()
    headers = df.columns.tolist()

    random.shuffle(data)
    index = int(len(data) * train_size)

    data_train = data[:index]
    data_test = data[index:]

    df_train = pd.DataFrame(data_train, columns=headers)
    df_test = pd.DataFrame(data_test, columns=headers)

    return df_train, df_test

# Corremos la función:
df_train, df_test = split_data(df=df_new, train_size=0.8, random_state=42)

df_train.head()

,DQO,SST,COLI,E_COLI,OD,TOX_D,TOX_V,SEMAFORO
0,14.010,10.00,2909.0,110.0,93.4,1.00,1.000,Amarillo
1,10.000,23.60,24196.0,1296.0,106.7,1.00,1.000,Amarillo
2,163.370,41.25,24000.0,11000.0,10.0,3.85,71.839,Rojo
3,10.565,10.00,2247.0,52.0,102.3,1.00,1.000,Amarillo
4,92.295,33.00,24000.0,4600.0,51.0,1.00,2.560,Rojo


### Modelo de regresión categórica

Ahora sí, contruya un modelo de regresión para predecir la probabilidad de ser clasificado con un color de semáforo en función de las variables predictoras.

#### Actividad 3

**Actividad**: Construya un modelo completo con las variables predictoras. Aplique las transformaciones que crea necesarias en función de la Actividad anterior. Si lo considera necesario para mejorar el modelo, puede incluir interacciones.

*NOTA*: El modelo de regresión categórica, similar al logístico, corresponde a la familia "categorical" en `Bambi`.

*NOTA*: Puede ser muy importante hacer algo para lograr la convergencia de las cadenas y mejorar la velocidad para generar las simulaciones.

In [ ]:
# Definir el modelo
# CODIGO DEL ALUMNO ############################################################
model_sem =



################################################################################

model_sem.build()
model_sem.graph()

In [ ]:
# Hago la inferencia estadística
results_sem = model_sem.fit(chains=4,draws=2000)

# Diagnóstico de la Cadena
az.plot_trace(results_sem, compact=False);
plt.tight_layout()
plt.show()

# Resumen de la inferencia
az.summary(results_sem)

### Análisis del modelo

#### Actividad 4

**Actividad**: En base a las distribuciones posteriores de los parámetros mostrada abajo, ¿qué variables parecen influir más? ¿Qué variables parecen aportar poca información al modelo? Para las variables categóricas, indique el significado de los parámetros, incluyendo cualquier categoría de referencia si la hay.

In [ ]:
# Grafico posterior de los parámetros
az.plot_forest( results_sem, figsize=(10,4), var_names=['~Intercept'],
                kind='ridgeplot', combined=True,
                ridgeplot_quantiles=[.05, .5, .95], ridgeplot_overlap=1, ridgeplot_truncate=False)
plt.show()

**Su respuesta**:

#### Actividad 5

**Actividad**: Simule valores de calificaciones de agua para el dataset de Testeo `df_test`. y obtenga la posterior predictiva como la categoría más elegida para cada dato.

In [ ]:
# Predicciones
# CODIGO DEL ALUMNO ############################################################



y_pps = []
# Posterior Predictiva


################################################################################

y_pps = np.array(y_pps)
y_pps.shape

Con estas simulaciones vamos a crear una matriz de confusión, que indica cuántas muestras se calificaron correctamente y cuáles no. La diagonal de la matriz son muestras calificadas correctamente.

In [ ]:
# Valores reales
y_true = df_test['SEMAFORO']

# Creamos un nuevo DataFrame
df_results = pd.DataFrame({'Actual': y_true, 'Predicted': y_pps})
df_results['Predicted']= df_results['Predicted'].astype("category")
df_results['Predicted'] =df_results['Predicted'].cat.rename_categories({0:'Amarillo',1:'Rojo',2:'Verde'})

# Calculamos la matriz de confusión
confusion_matrix2 = pd.crosstab(df_results['Actual'], df_results['Predicted'], rownames=['Actual'], colnames=['Predicted'])
confusion_matrix2

Predicted,Amarillo,Rojo,Verde
Actual,,,
Amarillo,279,10,3
Rojo,8,209,4
Verde,14,4,80


#### Actividad 6

**Actividad**: Utilizando la matriz de confusión (o la tablas `df_results`) calculada anteriormente, obtenga para el color **verde**:

- La precisión:
$$\frac{TP + TN}{TP + TN + FP + FN}$$
- La sensibilidad:
$$\frac{TP}{TP + FN}$$
- La especificidad:
$$\frac{TN}{TN + FP}$$

In [ ]:
# Calculamos FN, TN, TP y FP
# CODIGO DEL ALUMNO ############################################################






Acc =
Sen =
Spe =

################################################################################
print('Precision (Modelo 1)',round(Acc*100,2),'%')
print('Sensibilidad (Modelo 1)',round(Sen*100,2),'%')
print('Especificidad (Modelo 1)',round(Spe*100,2),'%')

Precision (Modelo 1) 95.91 %
Sensibilidad (Modelo 1) 81.63 %
Especificidad (Modelo 1) 98.64 %


## Ejercicio 2: Cierre del TP Integrador

El último ejercicio del trabajo práctico (y de todos los TPs de acá en más) es el avance del TP integrador. En este caso, el objetivo es cerrar el trabajo final, hallando el mejor modelo que se ajuste a los datos, y contestando la pregunta original formulada en el TP0.

### Actividad 1

**Actividad**: Seleccione el mejor modelo para sus datos, y conteste la pregunta de interés. Muestre claramente los modelos estudiados, y cómo eligió el modelo.

Para el modelo elegido, muestre detalladamente sus parámetros, el significado de sus parámetros y gráficos que muestren su funcionamiento y cómo ajusta los datos.

Utilice su modelo para dar respuesta a los interrogantes planteados al inicio del trabajo práctico final, utilizando ejemplos y sacando conclusiones.

Resuma aquí el trabajo realizado.

### Actividad 2

**Actividad**: Construya un poster en tamaño *A0* que resuma el trabajo realizado. El mismo debiera contenter la motivación y pregunta a contestar, el EDA, las características del modelo adoptado y la respuesta a las preguntas como resultado de la aplicación del modelo, con un sector de conclusiones.

El mismo será defendido durante la última jornada de la materia en una sesión de posters.

# Entrega del Trabajo Práctico

Para la entrega del TP3, siga los siguientes pasos:
1. Vuelva a correr el Colab desde 0 para asegurar que no haya ningún bug y todo funcione tal como se desea (`Disconnect and delete runtime`,`run all`)
2. Revise que todos los resultados estén presentes y como ustedes esperaban.
3. Asegúrese haber contestado todas la preguntas que requieren prosa.
4. Descargue el arcihvo .ipynb en su computadora

Revise que está todo como les gusta y ¡ya está listo! Ustedes son responables de que la entrega refleje sus intenciones, más allá de que nosotros podamos correr su archivo para revisar.

Los datos presentados abajo intentan ser para ustedes (y para nosotros) una verificación de que efectivamente han hecho la revisión final.

In [ ]:
print('Inicio de corrida: ',start_datetime)

end_datetime = datetime.datetime.now()
print('Fin de corrida: ',end_datetime)

elapsed_time = end_datetime-start_datetime
print('Tiempo de ejecución: ',round(elapsed_time.total_seconds()/60,1),'minutos')

Inicio de corrida:  2025-11-17 15:33:03.534309
Fin de corrida:  2025-11-17 15:43:54.759875
Tiempo de ejecución:  10.9 minutos
